# Structures with Unique Functionality

Looking in `buildings.yaml` you will see that many structures have unique properties. 
* on_cycle: action/s to be taken on each building renew cycle
* on_complete: action to be taken when the building is complete

These are generated during the `factionBuildingTimer`  function, where event messages are generated. Those messages are resolved in `structures.process_structure()` within the `resolveActionEvents` function. That meas that they are all resolved asynchonously and without dependance on each other. 


### A demonstration

In [1]:
import os
import  ssl, asyncio
import nest_asyncio
import pandas as pd

# moving back to the root, but idempotent in case we are already there
if 'function_app.py' not in os.listdir():
    os.chdir('../..')
print([f for f in os.listdir() if f == 'function_app.py'])

import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logging.info("logs showing as print")



ssl._create_default_https_context = ssl._create_unverified_context
asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())
# this is required for running in a Jupyter Notebook. 
nest_asyncio.apply()

INFO:root:logs showing as print


['function_app.py']


In [2]:
import function_app as f

executing local windows deployment
something wrong with your query: <class 'Exception'>


## The Observatory

```
  observatory:
    type: observatory
    label: building
    name: Stellar Observatory
    description: "Scans the solar system for planets, moons, comets,and asteroids."
    owned_by: pop
    faction_augments: 
      wealth: -10
    requires_attr: 
      industry: .7
    has_buttons:
      - study_object
    on_cycle:
      - scan_system
    effort: 5
```

The function of the timer function is only to get a list of messages that it will send to eventhub. It doesn't actually _do_ anything, it just creates actions for the `resolveActionEvents` to resolve. 

In [3]:
structure_messages = f.get_structure_messages()
pd.DataFrame(structure_messages)

DEBUG:asyncio:Using selector: SelectSelector
DEBUG:asyncio:Using selector: SelectSelector
INFO:root:EXOADMIN: number of items: 4
INFO:root:EXOADMIN: Total Messages generated: 4 at: < time at UTU:1084 >


,faction,pop,structure,action
0,"{'wealth': 3, 'name': 'Ponhenhalgou', 'objid':...","{'isIdle': 'true', 'name': 'Ponhenhalgou Tubas...","{'name': 'Farmland', 'objid': '1882898224757',...",structure
1,"{'wealth': '0', 'name': 'Quetylaban', 'objid':...","{'isIdle': 'true', 'name': 'Quetylaban Rez', '...","{'name': 'Industrial manufacturing complex', '...",structure
2,"{'wealth': '0', 'name': 'Quetylaban', 'objid':...","{'isIdle': 'true', 'name': 'Quetylaban Saufloc...","{'name': 'Shipyard', 'objid': '6129816944130',...",structure
3,"{'wealth': '0', 'name': 'Quetylaban', 'objid':...","{'isIdle': 'true', 'industry': '0.9', 'name': ...","{'name': 'Stellar Observatory', 'objid': '4325...",structure


In [4]:
obervatory_message = structure_messages[3]
obervatory_message['structure']

{'name': 'Stellar Observatory',
 'objid': '4325841147687',
 'ownedBy': '7292775901561',
 'userguid': '8d5b667f-b225-4641-b499-73b77558ff86',
 'type': 'observatory',
 'description': 'Scans the solar system for planets',
 'moons': 'None',
 'comets': 'None',
 'and asteroids.': 'None',
 'owned_by': 'pop',
 'faction_augments': '{wealth: -10}',
 'requires_attr': '{industry: 0.7}',
 'on_cycle': '[scan_system]',
 'effort': 5,
 'objtype': 'building',
 'id': '4325841147687'}

## Resolving those messages

First the timer function will query th structure to generate the discovery message. 

In [5]:
discovery_message = f.process_action_event_message(obervatory_message)
discovery_message


DEBUG:asyncio:Using selector: SelectSelector
INFO:root:EXOADMIN: processing message: structure at UTU:< time at UTU:1084 >
INFO:root:EXOADMIN: process_structure, structure: Stellar Observatory: 4325841147687
INFO:root:EXOADMIN: this structure augments a property of the faction
INFO:root:EXOADMIN: wealth has changed from 0 to 0
DEBUG:asyncio:Using selector: SelectSelector
INFO:root:EXOADMIN: this structure has a cyclical effect: ['scan_system']
INFO:root:EXOADMIN: processing cycle effect: scan_system
DEBUG:asyncio:Using selector: SelectSelector
INFO:root:EXOADMIN: query_counts_of_objects: {'planet': 4, 'moon': 10, 'star': 1, 'asteroid': 1}
INFO:root:EXOADMIN: found a star with probability 1.0 and roll 0.9501330980564693
INFO:root:EXOADMIN:       -------And with that processed STRUCTURE: {'faction': {'wealth': '0', 'name': 'Quetylaban', 'objid': '7667867938373', 'lat': -0.116, 'long': 0, 'infrastructure': 0, 'userguid': '8d5b667f-b225-4641-b499-73b77558ff86', 'objtype': 'faction', 'id': 

[{'action': 'discovery',
  'agent': {'name': 'Stellar Observatory',
   'objid': '4325841147687',
   'ownedBy': '7292775901561',
   'userguid': '8d5b667f-b225-4641-b499-73b77558ff86',
   'type': 'observatory',
   'description': 'Scans the solar system for planets',
   'moons': 'None',
   'comets': 'None',
   'and asteroids.': 'None',
   'owned_by': 'pop',
   'faction_augments': '{wealth: -10}',
   'requires_attr': '{industry: 0.7}',
   'on_cycle': '[scan_system]',
   'effort': 5,
   'objtype': 'building',
   'id': '4325841147687'},
  'object': 'star'}]

Then the event function will 'discover' or create the new celestial body. 

In [7]:
f.process_action_event_message(discovery_message[0])

DEBUG:asyncio:Using selector: SelectSelector
INFO:root:EXOADMIN: processing message: discovery at UTU:< time at UTU:1084 >
INFO:root:EXOADMIN: structure_system_query: 
    g.V().has('objid','4325841147687').in('owns').out('inhabits').out('isIn').valuemap()
    
DEBUG:asyncio:Using selector: SelectSelector
INFO:root:EXOADMIN: discovery has been made in system: {'objid': '9726813013638', 'name': 'Raste', 'class': 'ordered', 'isHomeSystem': 'true', 'glat': 39.465, 'glon': 2.219, 'gelat': 7.672, 'userguid': '8d5b667f-b225-4641-b499-73b77558ff86', 'objtype': 'system', 'id': '9726813013638'}
INFO:root:EXOADMIN: Creating star: <asteroid: p-type; 0044445742095; He>
DEBUG:asyncio:Using selector: SelectSelector
DEBUG:asyncio:Using selector: SelectSelector
INFO:root:EXOADMIN: Uploading data: {'nodes': [{'name': 'He', 'class': 'p-type', 'objid': '0044445742095', 'label': 'asteroid', 'pop_cap': 100, 'orbitsDistance': 3.258, 'mass': 3.0753260803374276e-07, 'radius': 0.00016668824341021442, 'isPopula

[]